<a href="https://colab.research.google.com/github/Navam16/Course-Recommendation/blob/main/course_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary libraries
!pip install scikit-surprise numpy==1.23.5


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.8 MB/s eta 0:00:00
ERROR: Operation cancelled by user
^C


In [ ]:
!pip install --upgrade --force-reinstall numpy pandas scikit-learn surprise


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.9 MB/s eta 0:00:00
  Using cached scikit_surprise-1.1.4.tar.gz (154 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 91.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 93.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from surprise import SVD, KNNBasic, Dataset, Reader
from surprise.model_selection import cross_validate
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, f1_score, confusion_matrix



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

ImportError: numpy.core.multiarray failed to import (auto-generated because you didn't call 'numpy.import_array()' after cimporting numpy; use '<void>numpy._import_array' to disable if you are certain you don't need it).

In [ ]:
 df = pd.read_csv('Coursera.csv')

NameError: name 'pd' is not defined

In [ ]:
# Ensure all text columns are properly encoded
df['Course Description'] = df['Course Description'].astype(str).apply(lambda x: x.encode('utf-8', 'ignore').decode('utf-8'))
df['Skills'] = df['Skills'].astype(str).apply(lambda x: x.encode('utf-8', 'ignore').decode('utf-8'))

# Convert Course Rating to numeric
df['Course Rating'] = pd.to_numeric(df['Course Rating'], errors='coerce')

# Drop missing values
df = df.dropna()

# Verify the cleaned data
print(df.info())
print(df.head())



<class 'pandas.core.frame.DataFrame'>
Index: 3440 entries, 0 to 3521
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Course Name         3440 non-null   object 
 1   University          3440 non-null   object 
 2   Difficulty Level    3440 non-null   object 
 3   Course Rating       3440 non-null   float64
 4   Course URL          3440 non-null   object 
 5   Course Description  3440 non-null   object 
 6   Skills              3440 non-null   object 
dtypes: float64(1), object(6)
memory usage: 215.0+ KB
None
                                         Course Name  \
0  Write A Feature Length Screenplay For Film Or ...   
1  Business Strategy: Business Model Canvas Analy...   
2                      Silicon Thin Film Solar Cells   
3                               Finance for Managers   
4       Retrieve Data using Single-Table SQL Queries   

                  University Difficulty Level  Course Rating  

In [ ]:
# Convert difficulty level into categorical codes
df['Difficulty Level'] = df['Difficulty Level'].astype('category').cat.codes

# Convert University column into categorical codes
df['University'] = df['University'].astype('category').cat.codes

# Convert 'Skills' into a list of keywords
df['Skills'] = df['Skills'].apply(lambda x: x.split() if isinstance(x, str) else [])

# Check the processed data
print(df.head())


                                         Course Name  University  \
0  Write A Feature Length Screenplay For Film Or ...          71   
1  Business Strategy: Business Model Canvas Analy...          25   
2                      Silicon Thin Film Solar Cells         178   
3                               Finance for Managers          52   
4       Retrieve Data using Single-Table SQL Queries          25   

   Difficulty Level  Course Rating  \
0                 1            4.8   
1                 1            4.8   
2                 0            4.1   
3                 3            4.8   
4                 1            4.6   

                                          Course URL  \
0  https://www.coursera.org/learn/write-a-feature...   
1  https://www.coursera.org/learn/canvas-analysis...   
2  https://www.coursera.org/learn/silicon-thin-fi...   
3  https://www.coursera.org/learn/operational-fin...   
4  https://www.coursera.org/learn/single-table-sq...   

                         

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# TF-IDF Vectorization on Course Descriptions
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['Course Description'])

# Compute the Cosine Similarity Matrix
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Function to recommend similar courses
def recommend_courses(course_title, num_recommendations=5):
    idx = df[df['Course Name'].str.contains(course_title, case=False, na=False)].index
    if len(idx) == 0:
        return "Course not found"

    idx = idx[0]  # Take first match
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:num_recommendations+1]

    course_indices = [i[0] for i in sim_scores]
    return df.iloc[course_indices][['Course Name', 'University', 'Course Rating']]

# Example: Get recommendations for a course
print(recommend_courses("Finance for Managers"))


                                            Course Name  University  \
1839  Fundamentals of financial and management accou...          93   
1891        Accounting and Finance for IT professionals          56   
1985                Introduction to Finance: The Basics         147   
419                  Finance for Non-Financial Managers          34   
1164                       Corporate Finance Essentials          52   

      Course Rating  
1839            4.7  
1891            4.5  
1985            4.6  
419             4.2  
1164            4.8  


In [ ]:
!pip install scikit-surprise

from surprise import SVD, Dataset, Reader
from surprise.model_selection import cross_validate

# Prepare the data for Surprise
reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(df[['Course Name', 'University', 'Course Rating']], reader)

# Train SVD model
svd = SVD()
cross_validate(svd, data, cv=5, verbose=True)


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.3191  0.3847  0.2979  0.3480  0.2939  0.3287  0.0339  
MAE (testset)     0.2131  0.2336  0.2075  0.2213  0.2051  0.2161  0.0104  
Fit time          0.04    0.04    0.06    0.04    0.05    0.05    0.01    
Test time         0.00    0.00    0.00    0.00    0.00    0.00    0.00    


{'test_rmse': array([0.31911239, 0.38471303, 0.29789472, 0.34795564, 0.29393323]),
 'test_mae': array([0.21312072, 0.23355998, 0.20753588, 0.22132824, 0.2050541 ]),
 'fit_time': (0.04284024238586426,
  0.0445404052734375,
  0.05904865264892578,
  0.04378247261047363,
  0.04876422882080078),
 'test_time': (0.0032460689544677734,
  0.002599954605102539,
  0.0032858848571777344,
  0.0027937889099121094,
  0.0025484561920166016)}

In [ ]:
!pip install scikit-learn pandas numpy


In [ ]:


# Ensure all columns are strings before concatenation
df["combined_features"] = (
    df["Course Name"].astype(str) + " " +
    df["Course Description"].astype(str) + " " +
    df["Skills"].astype(str)
)

# Convert text to numerical vectors using TF-IDF
vectorizer = TfidfVectorizer(stop_words="english")
tfidf_matrix = vectorizer.fit_transform(df["combined_features"])

# Compute cosine similarity between all courses
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Function to recommend courses
def recommend_courses(selected_indices, df, cosine_sim):
    if not selected_indices:
        return "No courses selected. Please choose at least one course."

    # Compute average similarity scores
    sim_scores = sum(cosine_sim[selected_indices]) / len(selected_indices)

    # Sort courses based on similarity scores (excluding already taken ones)
    sorted_courses = sorted(enumerate(sim_scores), key=lambda x: x[1], reverse=True)
    recommended_courses = [df.iloc[i[0]]["Course Name"] for i in sorted_courses if i[0] not in selected_indices][:5]

    return recommended_courses

# Display course options for the user
print("\n📋 Available Courses:")
for idx, course in enumerate(df["Course Name"][:20], 1):  # Show first 20 courses
    print(f"{idx}. {course}")

# Ask user to select courses by number
selected_indices = input("\nEnter the numbers of courses you have taken (comma-separated): ")
selected_indices = [int(i.strip()) - 1 for i in selected_indices.split(",") if i.strip().isdigit()]

# Get recommendations
recommendations = recommend_courses(selected_indices, df, cosine_sim)

# Display recommendations
print("\n📚 Recommended Courses for You:")
for i, course in enumerate(recommendations, 1):
    print(f"{i}. {course}")



📋 Available Courses:
1. Write A Feature Length Screenplay For Film Or Television
2. Business Strategy: Business Model Canvas Analysis with Miro
3. Silicon Thin Film Solar Cells
4. Finance for Managers
5. Retrieve Data using Single-Table SQL Queries
6. Building Test Automation Framework using Selenium and TestNG
7. Doing Business in China Capstone
8. Programming Languages, Part A
9. The Roles and Responsibilities of Nonprofit Boards of Directors within the Governance Process
10. Business Russian Communication. Part 3
11. Agile Projects:  Developing Tasks with Taiga
12. Esports Management Capstone Project
13. Hacking and Patching
14. Business Statistics and Analysis Capstone
15. Grab Data Fast with Vertical and Horizontal LOOKUP
16. Global Health: An Interdisciplinary Overview
17. Python Programming Essentials
18. Creating Dashboards and Storytelling with Tableau
19. Parallel programming
20. Recommendation Systems with TensorFlow on GCP

Enter the numbers of courses you have taken (comm